In [2]:
import json
import requests
from bs4 import BeautifulSoup
import re

In [3]:
var_dict = {
    "Type": "type",
    "Title": "title",
    "Other Titles": "otherTitles",
    "Author(s)/Inventor(s)": "author",
    "Advisor": "advisor",
    "Co-advisor": "coAdvisor",
    "Abstract": "abstract",
    "AbstractEnglish": "abstractEnglish",
    "Keywords": "keywords",
    "Subject CNPq": "subjectCnpq",
    "Program": "program",
    "Production unit": "productionUnit",
    "Publisher": "publisher",
    "Issue Date": "issueDate",
    "Publisher country": "publisherCountry",
    "Language": "language",
    "Right access": "rightAccess",
    "Appears in Collections": "appearsInCollections",
}

def get_pdf(url):
    url = url.replace('http://hdl.handle.net/', 'https://pantheon.ufrj.br/handle/')
    response = requests.get(url)
    html = response.content
    
    soup = BeautifulSoup(html, 'html.parser')

    # acha o link do pdf usando regex pelo grupo
    regex = 'href="(\/bitstream\/.*?\.pdf)"'
    matches = re.search(regex, soup.decode())

    pdf_url = None
    if matches:
        pdf_url = 'https://pantheon.ufrj.br' + matches.group(1)

    # Extract the URL
    if pdf_url:
        print(f"PDF URL: {pdf_url}")
    else:
        print("PDF URL not found.")

    # Find the table rows
    rows = soup.find('table', class_='itemDisplayTable').find_all('tr')

    # Create a dictionary to store the data
    data_dict = {}
    abstract = False

    # Loop through rows and extract key-value pairs
    for row in rows:
        label = row.find('td', class_='metadataFieldLabel')
        value = row.find('td', class_='metadataFieldValue')
        
        if label and value:
            label_text = label.get_text(strip=True).rstrip(':')
            value_text = value.get_text(strip=True)

            if label_text == "Abstract":
                if abstract:
                    data_dict["abstractEnglish"] = value_text
                else:
                    data_dict["abstract"] = value_text
                    abstract = True
            else:
                data_dict[var_dict[label_text]] = value_text

    data_dict["pdfUrl"] = pdf_url
    return data_dict

In [9]:
data = []
num = 2500

# abre doutorados.json
with open('doutorados.json') as doutorados:
    # carrega o arquivo json em um dicionário
    doutorados = json.load(doutorados)

    for doutorado in doutorados:
        if num == 0:
            break
        url = doutorado['urls'][0]
        try:
            info = get_pdf(url)
            print(doutorado['subjectsPOR'])
            info['subjectsPOR'] = [subjectPOR[0] for subjectPOR in doutorado['subjectsPOR']]
            # join array info['subjectsPOR] whith ','
            info['keywords'] = ','.join(info['subjectsPOR'])
            info['subjectCnpq'] = info['subjectCnpq'].replace('::', ',')
            data.append(info)
            num -= 1
        except:
            print("Erro ao obter dados do pdf")
            continue

with open('parsed_doutorados.json', 'w') as doutorados:
    # parse data to utf-8 json file
    data = json.dumps(data, indent=4, ensure_ascii=True).encode('utf8')
    doutorados.write(data.decode())


PDF URL: https://pantheon.ufrj.br/bitstream/11422/11466/3/BrunaReisLeiteFranca.pdf
[['Oscilações Subinerciais'], ['Ondas de Plataforma'], ['Ondas Con nadas Costeiras'], ['Maré Meteorológica'], ['Frentes Frias'], ['Fluxos de Calor'], ['Plataforma Continental Sudeste do Brasil']]
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11560/1/887655.pdf
[['Engenharia elétrica'], ['Codificação de vídeo de dupla finalidade'], ['HEVC']]
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11567/1/887747.pdf
[['Engenharia elétrica'], ['Futebol americano'], ['Modelos de ranqueamento'], ['Ranqueamento Bayesiano'], ['Ciência do esporte']]
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11570/1/887764.pdf
[['Engenharia elétrica'], ['Concessões de transmissão'], ['Leilões combinatórios'], ['Planejamento da expansão da transmissão']]
PDF URL: https://pantheon.ufrj.br/bitstream/11422/11571/1/887776.pdf
[['Engenharia elétrica'], ['Redes neurais'], ['Filtragem online'], ['Fusão de dados'], ['Aprendizado con